In [11]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from SALib.analyze import sobol
import warnings
from ema_workbench.analysis import feature_scoring
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
import ema_workbench.em_framework.optimization
from ema_workbench.em_framework.optimization import to_problem, epsilon_nondominated, EpsilonProgress, ArchiveLogger
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
    perform_experiments,
    Samplers,
)


from dike_model_function import DikeNetwork
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time
from ema_workbench.analysis import prim
from ema_workbench.em_framework.parameters import Constant
import pickle
warnings.filterwarnings("ignore")
ema_logging.log_to_stderr(ema_logging.INFO)


<Logger EMA (DEBUG)>

## Delft Blue Incoming Files

In [2]:
# Model
dike_model, planning_steps = get_model_for_problem_formulation(2)

In [3]:
# Load results from DB
scenario_results = []
seeds = [27097, 567] # 45646, 90, 676465]  # seeds used un DB - to change later
for seed in seeds:
    df = pd.read_csv(f"output/results_seed_{seed}.csv")  # Use '/'
    scenario_results.append(df)
    l = df.columns.tolist()
len(l)

37

In [4]:
# Load archive
archive_results = []
seeds = [27097, 567] # 45646, 90, 676465]  # seeds used un DB - to change later
for seed in seeds:
   archives = ArchiveLogger.load_archives(f"output/seed_{seed}_archive.tar.gz")  # Use '/'
   #for key in archives.keys():
   archive_results.append(archives)

In [5]:
pol = []
for policy in dike_model.levers:
    pol.append(policy)

for outcome in dike_model.outcomes:
    pol.append(outcome)

len(pol)
dike_model_problem =dike_model
dike_model_problem.outcomes

## Merge Seeds

In [6]:
for archive in archive_results:
    for key, df in archive.items():
        for col in dike_model.levers:
            if col.name not in df.columns:
                df[col.name] = 0

In [7]:
#Make sure that the results contain all policies (rest is set to 0)
for df in scenario_results:
    for col in dike_model.levers:
        if col.name not in df.columns:
            df[col.name] = 0

In [8]:
#Defining the problem
problem = to_problem(dike_model_problem, searchover="levers")

In [14]:
#Merge the results from the different seeds for each of the two scenarios
epsilons = [0.05] * len(dike_model.outcomes)

#Merge the list of results into a single set of non dominated results using the provided epsilon values
merged_results = epsilon_nondominated(scenario_results,epsilons,problem)

EMAError: Parameter names ['0_RfR 0', '0_RfR 1', '0_RfR 2', '1_RfR 0', '1_RfR 1', '1_RfR 2', '2_RfR 0', '2_RfR 1', '2_RfR 2', '3_RfR 0', '3_RfR 1', '3_RfR 2', '4_RfR 0', '4_RfR 1', '4_RfR 2', 'A.1_DikeIncrease 0', 'A.1_DikeIncrease 1', 'A.1_DikeIncrease 2', 'A.2_DikeIncrease 0', 'A.2_DikeIncrease 1', 'A.2_DikeIncrease 2', 'A.3_DikeIncrease 0', 'A.3_DikeIncrease 1', 'A.3_DikeIncrease 2', 'A.4_DikeIncrease 0', 'A.4_DikeIncrease 1', 'A.4_DikeIncrease 2', 'A.5_DikeIncrease 0', 'A.5_DikeIncrease 1', 'A.5_DikeIncrease 2'] not found in archive